In [1]:
import boto3
import pandas as pd

In [2]:


s3 = boto3.client('s3')
s3.download_file('data-handling-public', 'products.csv',
                 'cred/products.csv')

In [3]:
aws_bucket ='data-handling-public'
s3_key= 'products.csv'
local_path= 'cred/products.csv'

In [4]:

response = s3.get_object(Bucket=aws_bucket, Key=s3_key)

status = response.get("ResponseMetadata", {}).get("HTTPStatusCode")

if status == 200:
    print(f"Successful S3 get_object response. Status - {status}")
    products_df = pd.read_csv(response.get("Body"))
    
else:
    print(f"Unsuccessful S3 get_object response. Status - {status}")

Successful S3 get_object response. Status - 200


In [5]:
products_df

,Unnamed: 0,product_name,product_price,weight,category,EAN,date_added,uuid,removed,product_code
0,0,FurReal Dazzlin' Dimples My Playful Dolphin,£39.99,1.6kg,toys-and-games,7425710935115,2005-12-02,83dc0a69-f96f-4c34-bcb7-928acae19a94,Still_avaliable,R7-3126933h
1,1,Tiffany's World Day Out At The Park,£12.99,0.48kg,toys-and-games,487128731892,2006-01-09,712254d7-aea7-4310-aff8-8bcdd0aec7ff,Still_avaliable,C2-7287916l
2,2,Tiffany's World Pups Picnic Playset,£7.00,590g,toys-and-games,1945816904649,1997-03-29,b089ef6f-b628-4e37-811d-fffe0102ba64,Still_avaliable,S7-1175877v
3,3,Tiffany's World Wildlife Park Adventures,£12.99,540g,toys-and-games,1569790890899,2013-03-20,d55de422-8b98-47d6-9991-e4bc4c5c0cb0,Removed,D8-8421505n
4,4,Cosatto Cosy Dolls Pram,£30.00,1.91kg,toys-and-games,7142740213920,2007-12-23,7945b657-cb02-4cc5-96cf-f65ed0a8f235,Still_avaliable,B6-2596063a
...,...,...,...,...,...,...,...,...,...,...
1848,1848,Goodmans Qi Autosense In Car Phone Holder,£15.00,125g,diy,8185827751600,2020-08-28,3ff21cb1-8e90-499d-b5a0-6839fb83e70f,Still_avaliable,x4-2227164k
1849,1849,Goodmans Qi Wireless Charging Phone Stand,£15.00,134g,diy,9769724533935,2012-05-20,a7910f74-c2b6-4144-98b7-30ca68f34d2d,Still_avaliable,i3-8338545Z
1850,1850,RAC 12V Wet & Dry Vacuum,£18.00,820g,diy,3113855860327,1993-11-28,66d3efa0-f480-4da9-893b-e8aaff53e27b,Still_avaliable,n4-1642658b
1851,1851,Goodmans Qi Autosense In Car Phone Holder,£15.00,125g,diy,9195087467400,2016-09-20,efb472f1-8f00-448e-a874-9fa9812eeab5,Still_avaliable,A3-7619070S


In [26]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1853 entries, 0 to 1852
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     1853 non-null   int64 
 1   product_name   1849 non-null   object
 2   product_price  1849 non-null   object
 3   weight         1849 non-null   object
 4   category       1849 non-null   object
 5   EAN            1849 non-null   object
 6   date_added     1849 non-null   object
 7   uuid           1849 non-null   object
 8   removed        1849 non-null   object
 9   product_code   1849 non-null   object
dtypes: int64(1), object(9)
memory usage: 144.9+ KB


In [7]:
import re




def convert_to_kg(weight):
    # Handle multiple units (e.g., "8 x 85g")
    multiple_match = re.match(r'(\d+)\s*x\s*(\d+\.?\d*)(\w+)', weight)
    if multiple_match:
        count = int(multiple_match.group(1))
        value = float(multiple_match.group(2))
        unit = multiple_match.group(3)
        weight_value = count * value
    else:
        # Extract value and unit
        match = re.match(r'(\d+\.?\d*)(\w+)', weight)
        if not match:
            return None
        weight_value = float(match.group(1))
        unit = match.group(2)

    # Conversion factors
    if unit == 'kg':
        return weight_value
    elif unit == 'g':
        return weight_value * 0.001
    elif unit == 'ml':
        return weight_value * 0.001  # Assuming 1:1 ratio for ml to g
    elif unit == 'oz':
        return weight_value * 0.0283495
    else:
        return None  # Unknown unit



In [8]:
products_df['date_added'] = pd.to_datetime(
    products_df['date_added'], format='mixed', errors='coerce')

In [29]:
products_df=products_df.dropna()

In [30]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1844 entries, 0 to 1852
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Unnamed: 0     1844 non-null   int64         
 1   product_name   1844 non-null   object        
 2   product_price  1844 non-null   object        
 3   weight         1844 non-null   object        
 4   category       1844 non-null   object        
 5   EAN            1844 non-null   object        
 6   date_added     1844 non-null   datetime64[ns]
 7   uuid           1844 non-null   object        
 8   removed        1844 non-null   object        
 9   product_code   1844 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(8)
memory usage: 158.5+ KB


In [31]:
products_df = products_df.convert_dtypes()

In [32]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1844 entries, 0 to 1852
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Unnamed: 0     1844 non-null   Int64         
 1   product_name   1844 non-null   string        
 2   product_price  1844 non-null   string        
 3   weight         1844 non-null   string        
 4   category       1844 non-null   string        
 5   EAN            1844 non-null   string        
 6   date_added     1844 non-null   datetime64[ns]
 7   uuid           1844 non-null   string        
 8   removed        1844 non-null   string        
 9   product_code   1844 non-null   string        
dtypes: Int64(1), datetime64[ns](1), string(8)
memory usage: 160.3 KB


In [33]:

# Apply conversion to the DataFrame
products_df['weight'] = products_df['weight'].apply(convert_to_kg)

In [34]:
https://data-handling-public.s3.eu-west-1.amazonaws.com/date_details.json

SyntaxError: invalid decimal literal (4019132870.py, line 1)

In [ ]:
type(products_weights)

pandas.core.series.Series

In [1]:
import requests

In [3]:
header = {"x-api-key": "yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX"}

In [4]:
url ='https://data-handling-public.s3.eu-west-1.amazonaws.com/date_details.json'

In [5]:
def retrieve_data( header,url):
    try:
            response = requests.get(url, headers=header)
            # This will raise an HTTPError for bad responses (4xx and 5xx)
            response.raise_for_status()

            # Process the response if no exceptions were raised
            data = response.json()  # Assuming the response is in JSON f

    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except requests.exceptions.ConnectionError as conn_err:
        print(f"Connection error occurred: {conn_err}")
    except requests.exceptions.Timeout as timeout_err:
        print(f"Timeout error occurred: {timeout_err}")
    except requests.exceptions.RequestException as req_err:
        print(f"An error occurred: {req_err}")
    except ValueError as json_err:
        print(f"JSON decoding failed: {json_err}")

    return data

In [6]:
json_data = retrieve_data(header=header,url=url)

In [7]:
import pandas as pd

In [8]:
date_df = pd.DataFrame.from_dict(json_data)

In [9]:
date_df['month']= pd.to_numeric(date_df['month'],errors='coerce')


In [10]:
date_df = date_df.dropna()

In [11]:
date_df['month'] = date_df['month'].astype('int64')

In [12]:
date_df['year'] = pd.to_datetime(
    date_df['year'], errors='coerce')

In [13]:
date_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 120123 entries, 0 to 120160
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   timestamp    120123 non-null  object        
 1   month        120123 non-null  int64         
 2   year         120123 non-null  datetime64[ns]
 3   day          120123 non-null  object        
 4   time_period  120123 non-null  object        
 5   date_uuid    120123 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 6.4+ MB


In [35]:
date_df['year']

0         2012
1         1997
2         1994
3         2001
4         2015
          ... 
120156    2022
120157    1997
120158    2011
120159    2013
120160    2008
Name: year, Length: 120161, dtype: object

In [14]:
import yaml
from sqlalchemy import create_engine
import psycopg2

In [24]:
with open('cred/pg_admin_creds.yaml') as file:
    db_cred = yaml.safe_load(file)

In [25]:
DATABASE_TYPE = 'postgresql'
DBAPI = 'psycopg2'
HOST = db_cred['RDS_HOST']
USER = db_cred['RDS_USER']
PASSWORD = db_cred['RDS_PASSWORD']
DATABASE = db_cred['RDS_DATABASE']
PORT = db_cred['RDS_PORT']
engine = create_engine(
    f"{DATABASE_TYPE}+{DBAPI}://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}")

In [26]:
try:
    date_df.to_sql('dim_date_times', engine,
                   index=False, if_exists='replace')
    print(f"Data uploaded to legacy_users successfully.")
except Exception as e:
    print(f"Error uploading data to legacy_users: {e}")

Data uploaded to legacy_users successfully.


In [34]:
import os
from dotenv import load_dotenv

In [35]:
load_dotenv()

False

In [42]:
aws_api_key = os.getenv('AWS_API_KEY')

In [43]:
print(aws_api_key)

yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX


In [44]:
header = {"x-api-key": aws_api_key}

In [45]:
print(header)

{'x-api-key': 'yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX'}
